In [1]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model

import numpy as np

import glob
import cv2

import matplotlib.pyplot as plt

# Load Data

In [2]:
list_test_image = glob.glob("C:/Users/PC_ML/Desktop/Gun_Knife_Censor/Dataset/guns-knives-yolo/guns-knives-yolo/test/images/*")
path_test_label = "C:/Users/PC_ML/Desktop/Gun_Knife_Censor/Dataset/guns-knives-yolo/guns-knives-yolo/test/labels/"
path_test_image = "C:/Users/PC_ML/Desktop/Gun_Knife_Censor/Dataset/guns-knives-yolo/guns-knives-yolo/test/images/"

In [3]:
def data_loader(list_train_image,path_train_label):
    list_train_x = []
    list_train_y_coff = []
    list_train_y_bbox = []
    
    for file_image in list_train_image:
        file_name = file_image.split("\\")[1].replace(".jpg","")
        file_label = path_train_label+file_name+".txt"
    
        #Read Image
        img = cv2.imread(file_image)
        #Resize
        img = cv2.resize(img, (300,300), interpolation = cv2.INTER_AREA)
        #Normalize Image
        img = img/255.0
        #Add to list
        list_train_x.append(img)
    
        #Read Label
        label = np.zeros((14,5)) # 14 max object in one image 5 is coff,x,y,w,h
        with open(file_label) as file:
            for i,line in enumerate(file.readlines()):
                _line = line.split()
                x = float(_line[1])
                y = float(_line[2])
                w = float(_line[3])
                h = float(_line[4])
                label[i] = np.array([1.0,x,y,w,h])
        #label = label.reshape(14*5)
        list_train_y_coff.append(label[:,0])
        list_train_y_bbox.append(label[:,1:].reshape(14*4))
    
    list_train_x = np.asarray(list_train_x)
    list_train_y_coff = np.asarray(list_train_y_coff)
    list_train_y_bbox = np.asarray(list_train_y_bbox)
    return list_train_x,list_train_y_coff,list_train_y_bbox

In [4]:
test_x,test_y_coff,test_y_bbox = data_loader(list_test_image,path_test_label)

In [5]:
print(f"test_x:{test_x.shape}")
print(f"test_y_coff:{test_y_coff.shape}")
print(f"test_y_bbox:{test_y_bbox.shape}")

test_x:(432, 300, 300, 3)
test_y_coff:(432, 14)
test_y_bbox:(432, 56)


# Load Model

In [6]:
model = load_model("model/model_final.keras")
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 300, 300, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 298, 298, 32  896         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 max_pooling2d (MaxPooling2D)   (None, 149, 149, 32  0           ['conv2d[0][0]']                 
                                )                                                             

# Predict

In [7]:
y_pred_coff,y_pred_bbox = model.predict(test_x)
y_pred_bbox = y_pred_bbox.reshape(y_pred_bbox.shape[0],14,4)
print(y_pred_coff)
print(y_pred_bbox)

14/14 [==============================] - 10s 40ms/step
[[9.9996579e-01 9.5351726e-01 4.9079618e-01 ... 4.8942585e-04
  5.4722134e-04 5.2704697e-04]
 [9.9999964e-01 1.2209723e-12 3.8481569e-13 ... 3.8553036e-10
  3.0209546e-09 2.9778711e-09]
 [9.9999809e-01 6.2538681e-12 1.1016961e-12 ... 2.6015584e-10
  1.3940593e-08 1.3077828e-08]
 ...
 [9.9999988e-01 4.3575029e-11 5.3047650e-08 ... 4.2002903e-12
  3.5575688e-11 4.1389999e-11]
 [9.9999940e-01 2.0486952e-07 5.6193858e-05 ... 2.6053323e-10
  6.0669336e-10 6.8887934e-10]
 [9.9999964e-01 3.5818902e-22 1.5532441e-15 ... 7.1661630e-14
  1.1670762e-10 1.4786974e-10]]
[[[0.38888127 0.40903983 0.38927814 0.34862494]
  [0.5000562  0.3811046  0.26350674 0.1570607 ]
  [0.3599424  0.31328648 0.06349866 0.05584588]
  ...
  [0.         0.         0.         0.        ]
  [0.         0.         0.         0.        ]
  [0.         0.         0.         0.        ]]

 [[0.45004928 0.48771286 0.6132166  0.5786258 ]
  [0.         0.         0.         0

In [8]:
print(y_pred_bbox.shape)
print(y_pred_bbox[0])

(432, 14, 4)
[[0.38888127 0.40903983 0.38927814 0.34862494]
 [0.5000562  0.3811046  0.26350674 0.1570607 ]
 [0.3599424  0.31328648 0.06349866 0.05584588]
 [0.41336077 0.2994592  0.063931   0.05884409]
 [0.         0.         0.03423524 0.        ]
 [0.         0.01790781 0.         0.        ]
 [0.         0.         0.03130925 0.        ]
 [0.18566883 0.         0.03028812 0.        ]
 [0.28536302 0.         0.07182331 0.        ]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]]


In [9]:
print(y_pred_coff.shape)
print(y_pred_coff[0])

(432, 14)
[9.9996579e-01 9.5351726e-01 4.9079618e-01 3.6416754e-01 6.5773720e-01
 5.7699136e-03 2.0518545e-03 6.3803503e-03 6.3779685e-03 3.7619614e-03
 6.3336955e-04 4.8942585e-04 5.4722134e-04 5.2704697e-04]


# Save Predict to txt file

In [10]:
th = 0.5
output_path = "predict/"

#Loop Each Sample
for i in range(y_pred_coff.shape[0]):

    file_name = list_test_image[i].split("\\")[1].replace(".jpg","")
    
    img =cv2.imread(list_test_image[i])

    img_h = img.shape[0]
    img_w = img.shape[1]
    
    #Loop Each Image
    #print("-"*100)
    list_bbox = []
    for j in range(y_pred_coff.shape[1]):
        coff = y_pred_coff[i][j]
        x = y_pred_bbox[i][j][0]
        y = y_pred_bbox[i][j][1]
        w = y_pred_bbox[i][j][2]
        h = y_pred_bbox[i][j][3]
        if(coff > th):
            list_bbox.append(f"0 {x:.6f} {y:.6f} {w:.6f} {h:.6f}")
            
            y = int(y*img_h)
            x = int(x*img_w)
            h = int(h*img_h)
            w = int(w*img_w)
            
            s_p = [x - w//2,y - h//2]
            e_p = [x + w//2,y + h//2]
            
            color = [0,0,255]
            thickness = 2
            img = cv2.rectangle(img, s_p, e_p, color, thickness)

            img = cv2.putText(img, f"{coff:.3}", s_p, cv2.FONT_HERSHEY_SIMPLEX, 1, color, thickness , cv2.LINE_AA)

    #cv2.imshow(file_name,img)
    out_img_path = output_path+file_name
    print(out_img_path)
    cv2.imwrite(f"{output_path}{file_name}.jpg",img)

    f = open(f"{output_path}\labels\{file_name}.txt", "w")
    f.write("\n".join(list_bbox))
    f.close()


    #key = cv2.waitKey(0) 
    #cv2.destroyAllWindows()
    #if(key == ord('q')):
    #    break
            

predict/--------_------_jpg.rf.4eb0868f6cd41827c921043ddfa37ff9
predict/--------_------_jpg.rf.73a95f4c2fe5cfa9c071b8f4d2a02139
predict/--------_------_jpg.rf.a34ba4c1ed5c93d2f9bb8289a50a4e46
predict/108
predict/147
predict/155
predict/164
predict/167
predict/220
predict/237
predict/275
predict/279
predict/289
predict/290
predict/292
predict/296
predict/37
predict/378
predict/423
predict/445_jpg.rf.2e04379013684f454abbc00564910fcc
predict/452
predict/460
predict/465
predict/480
predict/50
predict/503
predict/508
predict/515
predict/523
predict/83
predict/89
predict/ABbframe00277_jpg.rf.1b722bd1914427b42b9579722538945d
predict/ABbframe00430_jpg.rf.96d982419a6d9599251ac97550753706
predict/ABbframe00457_jpg.rf.13a55678a98891aae40e7f4006ff799f
predict/ABmframe00226_jpg.rf.e824fe8c2d23f8443bfe36490b8374b6
predict/ABsframe00019_jpg.rf.452d8624f6d5a67b14393b30c1aa38a9
predict/ABsframe00214_jpg.rf.ae6c1e7d4c8e474123569e8bfa08f701
predict/armas--1000-_jpg.rf.c1e88f70c300deecad23d14748667259
pre